# Manipulacion de bases de datos

Para la ciencia de datos es importante la obtencion, la estructuracion y la calidad de los datos. Entre los distintos formatos para manipular está las bases de datos, archivos CSV o archivos JSON.

En este caso voy a ocupar la manipulación de bases de datos en sqlite3 con una base de datos llamada `PreciosMundi.db` que está basado en los precios de la web [precios mundi](www.preciosmundi.com) que se dedica a comparar los precios de una canasta familiar en comun entre distintos paises, con el fin de comparar el costo de vida.

Precios mundi cuenta con varias tablas como *Paises, Precios, Productos, Continentes, Idiomas, Monedas*, cada una de estas tablas estan vinculadas unas a otras a traves de ids, pero esto lo explicaré mas adelante cuando se llegue al apartado de creación.

1. [Consulta](#consulta)
2. [Creacion](#creación)
3. [Actualizacion](#actualización)
4. [Borrado](#borrado)

## Consulta.

Este es quizas el apartado mas importante y mas complejo de todos, todos empiezan con la palabra `SELECT` y `FROM`, pero de momento no tenemos ninguna información de las tablas y sus columnas, para saberlo debemos usar la funcion `.schema` de sqlite para poderlo consultar.

```sql

-- Tabla Continentes
CREATE TABLE Continentes (id INTEGER PRIMARY KEY, continente TEXT);

-- Tabla Productos
CREATE TABLE Productos (
id INTEGER PRIMARY KEY,
producto TEXT,
categoria INTEGER,
FOREIGN KEY (categoria) REFERENCES Categorias (id)
);

-- Tabla Precios
CREATE TABLE Precios (id INTEGER PRIMARY KEY, country_id INTEGER, product_id INTEGER, price FLOAT, FOREIGN KEY (country_id) REFERENCES "Paises_temp" (id), FOREIGN KEY (product_id) REFERENCES Productos (id));

-- Tabla Idiomas
CREATE TABLE Idiomas (id INTEGER PRIMARY KEY, idioma TEXT);

-- Tabla Paises
CREATE TABLE Paises (id INTEGER PRIMARY KEY, pais TEXT, continente INTEGER, capital TEXT, poblacion INTEGER, idioma INTEGER, moneda INTEGER, FOREIGN KEY (continente) REFERENCES Continentes (id), FOREIGN KEY (idioma) REFERENCES Idiomas (id), FOREIGN KEY (moneda) REFERENCES Monedas (id));

-- Tabla Monedas
CREATE TABLE Monedas (id INTEGER PRIMARY KEY, nombre TEXT, enemonico TEXT);
```
Ya teniendo algo de informacion, podemos hacer las primeras consultas:
```sql
-- Seleccionar todo
SELECT * FROM Precios;
```
Esta es la salida de la tabla precios:
```sql
(1, 127, 1, 1.54)
(2, 127, 2, 1.72)
(3, 127, 3, 12.57)
(4, 127, 4, 5.09)
(5, 127, 5, 5.69)
(6, 127, 6, 8.53)
(7, 127, 7, 14.49)
(8, 127, 8, 3.24)
(9, 127, 9, 2.65)
(10, 127, 10, 3.04)...
```
```sql
-- Seleccionar una columna en particular
SELECT price FROM Precios;
```
salida:
```sql
(1.54,)
(1.72,)
(12.57,)
(5.09,)
(5.69,)
(8.53,)
(14.49,)
(3.24,)
(2.65,)
(3.04,)
(10.05,)...
```
```sql
-- Seleccionar varias columnas
SELECT price, product_id FROM Precios
```
salida:
```sql
(1.54, 1)
(1.72, 2)
(12.57, 3)
(5.09, 4)
(5.69, 5)
(8.53, 6)
(14.49, 7)
(3.24, 8)
(2.65, 9)
(3.04, 10)
(10.05, 11)...
```
Tambien puedo hacer una selección condicionada, aqui la clave es la palabra `WHERE`, donde propone una condición para filtrar el resultado.

```sql
SELECT * FROM Precios WHERE country_id = 10 LIMIT 5;
-- salida
(541, 10, 1, 0.79)
(542, 10, 2, 0.57)
(543, 10, 3, 5.98)
(544, 10, 4, 2.15)
(545, 10, 5, 2.27)
```
No solo se limita a buscar una igualdad con un numero entero, puede recibir condiciones como un valor mayor > o menor < o que comience con una palabra LIKE "palab%", tambien puede hacerse con operaciones:
```sql
-- Selecciona condicionado a que el precio sea mayor al promedio.
SELECT * FROM Precios WHERE price > (SELECT AVG(price) FROM Precios) LIMIT 5;
-- salida
(28, 127, 28, 27782.37)
(36, 127, 36, 1587.56)
(37, 127, 37, 1256.82)
(38, 127, 38, 2513.64)
(39, 127, 39, 1852.16)
```
Tambien podemos limitar las salidas de las consultas, para no tener una tira muy larga, esto lo hacemos para ver una previa de los resultados de la tabla, esto se hace con la instruccion `LIMIT`

```sql
-- Limitando la salida a 5 resultados
SELECT * FROM Precios LIMIT 5;
```
La salida imprime los primeros 5 resultados, es similar a la funcion `.head(5)` de un dataframe:
```sql
(1, 127, 1, 1.54)
(2, 127, 2, 1.72)
(3, 127, 3, 12.57)
(4, 127, 4, 5.09)
(5, 127, 5, 5.69)
```
Tambien podemos limitar a los ultimos 5 resultados, pero el resultado no es igual a la funcion `.tail(5)`, para eso se hace uso de la instruccion `ORDER BY` para ordenar y `DESC` para hacerlo de forma descendente, 

```sql
-- Seleccionando los ultimos 5 resultados
SELECT * FROM Precios ORDER BY id DESC LIMIT 5;

(4871, 126, 51, 3.86)
(4870, 126, 50, 2.19)
(4869, 126, 49, 11.57)
(4868, 126, 48, 21.85)
(4867, 126, 47, 33.42)
```
Estos datos asi como estan no se entienden, porque solo se ven un monton de numeros que les falta contexto, esto se debe a que los numeros hacen referencias a indices de otras tablas, para visualizarlo mejor, debemos fusionar esas tablas momentaneamente con las instruccion `JOIN ON`

```sql
-- Para presenttar la tabla de forma mas bonita:
SELECT Precios.id, Paises.pais, Productos.producto, Precios.price
FROM Precios
JOIN Paises ON Precios.country_id = Paises.id
JOIN Productos ON Precios.product_id = Productos.id;

-- Salida

(1, 'Australia', 'Leche 1lt', 1.54)
(2, 'Australia', 'Agua 1.5lt', 1.72)
(3, 'Australia', 'Botella de Vino - Calidad media', 12.57)
(4, 'Australia', 'Cerveza nacional 0.5lt', 5.09)
(5, 'Australia', 'Cerveza importada 33cl', 5.69)

```

## Creación e inserción.

Si quieres crear una tabla las instrucciones clave son `CREATE TABLE tabla_a_crear` un ejemplo es usar `.schema`, la salida de este comando es precisamente un comando de creacion de una tabla, perfectamente se puede copiar y pegar.

Lo importante es saber que tipo de datos elegir segun el alcance de los datos como la cantidadd maxima de caracteres, tambien si los datos obedecen a etiquetas o categorias etc:

SQLite3 es un gestor de base de datos muy limitado y muy liviano, no maneja ningun tipo de encriptacion, tampoco tiene una variedad amplia de tipos de datos, los datos disponibles son los siguientes:

### 2. Clases de Almacenamiento y Datatypes
Cada valor almacenado en una base de datos SQLite (o manipulado por el el motor de base de datos) tiene una de las siguientes clases de almacenamiento:

- **NULL**. El valor es un valor NULO y es el valor por defecto cuando creas una columna sin asignar tipo de valor.
- **ENTERO**. El valor es un entero firmado, almacenado en 0, 1, 2, 3, 4, 6 u 8 bytes dependiendo de la magnitud del valor.
- **REAL**. El valor es un valor de punto flotante, almacenado como un número de coma flotante IEEE de 8 bytes.
- **TEXTO**. El valor es una cadena de texto, almacenada usando el codificación de base de datos (UTF-8, UTF-16BE o UTF-16LE).
- **BLOB**. El valor es una gota de datos, almacenados exactamente como fue entrada.

Una clase de almacenamiento es más general que un tipo de datos. La clase de almacenamiento INTEGER, por ejemplo, incluye 7 enteros diferentes tipos de datos de diferentes longitudes. Esto hace la diferencia en el disco. Pero tan pronto como los valores INTEGER se leen fuera del disco y en la memoria para procesamiento, se convierten al tipo de datos más general (número entero firmado de 8 bytes). Y así, en su mayor parte, "clase de almacenamiento" es indistinguible de "datatype" y los dos términos se pueden usar indistintamente.

Cualquier columna en una base de datos SQLite versión 3, excepto un CLAVE PRIMARIA ENTERA columna, se puede utilizar para almacenar un valor de cualquier clase de almacenamiento.

Todos los valores en las sentencias SQL, ya sean literales incrustados en SQL texto de declaración o parámetros obligado a declaraciones SQL precompiladas tener una clase de almacenamiento implícito. En las circunstancias descritas a continuación, el el motor de base de datos puede convertir valores entre clases de almacenamiento numérico (INTEGER y REAL) y TEXT durante la ejecución de la consulta.

### 2.1. Datatype Booleano
SQLite no tiene una clase de almacenamiento booleana separada. En cambio, los valores booleanos se almacenan como enteros 0 (falso) y 1 (verdadero).

SQLite reconoce las palabras clave **"TRUE" y "FALSE"**, a partir de la versión 3.23.0 (2018-04-02), pero esas palabras clave son realmente solo ortografías alternativas para los literales enteros 1 y 0 respectivamente.

### 2.2. Fecha y hora Datatype
SQLite **no tiene una clase de almacenamiento reservada** para almacenar fechas y/o horas. En cambio, el **incorporado Funciones de Fecha y Hora de SQLite son capaces de almacenar fechas y horas como valores TEXT, REAL o INTEGER**:

- **TEXT:** como cadenas ISO8601 ("AAAA-MM-DD HH:MM:SS.SSS").
- **REAL:** como números del día de Julian, el número de días desde entonces mediodía en Greenwich el 24 de noviembre de 4714 a.C. según el calendario Gregoriano proléptico.
- **INTEGER** como Unix Time, la cantidad de segundos desde entonces 1970-01-01 00:00:00 UTC.
- 
Las aplicaciones pueden optar por almacenar fechas y horas en cualquiera de estos formatos y convertir libremente entre formatos utilizando el incorporado funciones de fecha y hora.

Tipo Afinidad
Los motores de bases de datos SQL que usan tipificación rígida generalmente intentarán convertir automáticamente los valores al tipo de datos apropiado. Considera esto:
```sql
CREATE TABLE t1 ( a INT, b VARCHAR ( 10 ) );
INSERT INTO t1 ( a, b ) VALUES ( '123', 456 );
```
La base de datos con tipo rígido convertirá la cadena '123' en una entero 123 y el entero 456 en una cadena '456' antes de haciendo el inserto.

Para maximizar la compatibilidad entre SQLite y otras bases de datos motores, y para que el ejemplo anterior funcione en SQLite como lo hace en otros motores de bases de datos SQL, SQLite admite el concepto de **"afinidad tipo"** en columnas. 

La afinidad de tipo de una columna es el tipo recomendado para los datos almacenados en esa columna. La idea importante aquí es que se recomienda el tipo, no requerido. 

**Cualquier columna aún puede almacenar cualquier tipo de datos**. Es solo que algunas columnas, dada la opción, preferirán usar una clase de almacenamiento sobre otra. La clase de almacenamiento preferida para una columna se llama su "afinidad".

A cada columna en una base de datos SQLite 3 se le asigna una de las siguientes afinidades de tipo:

- TEXT
- NUMERIC
- INTEGER
- REAL
- BLOB

> **Nota histórica:** La afinidad de tipo **"BLOB"** solía llamarse **"NINGUNO"**. Pero ese término era fácil de confundir con "sin afinidad" y así fue renombrado.

Una columna con afinidad TEXT almacena todos los datos usando clases de almacenamiento NULL, TEXT o BLOB. Si se insertan datos numéricos en una columna con afinidad TEXT se convierte en forma de texto antes de ser almacenado.

Aun hay mas contenido del tipo de afinidades, lo puede consultar en la web de [Sqlite](https://www.sqlite.org/datatype3.html)

## Asociaciones a otras tablas con claves foraneas:

Esto es una practica recomenda para mantener la consistencia de los datos y mantener bajo el consumo de memoria ram.

En que consiste:

Supongase que esté llenando una base de datos a mano:

```sql
(1, 'Australia', 'Leche 1lt', 1.54)
(2, 'Australía', 'Agua 1.5lt', 1.72)
(3, 'Aústralia', 'Botella de Vino - Calidad media', 12.57)
(4, 'Austral', 'Cerveza nacional 0.5lt', 5.09)
(5, 'australia', 'Cerveza importada 33cl', 5.69)
```
Se puede notar que fallo al escribir la palabra `Australia` generando asi inconsistencias, lo mejor en este caso es crear una tabla de `Paises`, donde tenga el nombre del pais bien escrito y mejor hago uso del id de la tabla `Paises` para referirme a `Australia`.

Las palabras clave aqui son las palabras FOREIGN KEY y REFERENCES, fijemonos en el esquema de la base de datos.

```sql

-- Tabla Continentes
CREATE TABLE Continentes (id INTEGER PRIMARY KEY, continente TEXT);
```
Fijemonos que la tabla continentes tiene un campo id que es del tipo entero con una llave primaria o `PRIMARY KEY`, esto hace que se rellene un numero consecutivo cada vez que se inserta un nuevo dato.

Ahora fijemonos en la tabla paises:
```sql
-- Tabla Paises
CREATE TABLE Paises (id INTEGER PRIMARY KEY, pais TEXT, continente INTEGER, capital TEXT, poblacion INTEGER, idioma INTEGER, moneda INTEGER, FOREIGN KEY (continente) REFERENCES Continentes (id), FOREIGN KEY (idioma) REFERENCES Idiomas (id), FOREIGN KEY (moneda) REFERENCES Monedas (id));
```
Entre los campos tiene `continente INTEGER` que indica que esta columna solo permite números enteros, pero ademas tiene `FOREIGN KEY (continente) REFERENCES Contientes(id)`, donde vincula la columna contiente como una clave foranea que hace referencia al id de la tabla Continentes.

a su vez Paises tiene un monton de otros datos vinculados como idioma y moneda, pero iniciamos este ejemplo con la mala escritura de la palabra Australia de la tabla Precios, vamos a analizar la tabla precios:

```sql
-- Tabla Precios
CREATE TABLE Precios (id INTEGER PRIMARY KEY, country_id INTEGER, product_id INTEGER, price FLOAT, FOREIGN KEY (country_id) REFERENCES Paises (id), FOREIGN KEY (product_id) REFERENCES Productos (id));
```
La tabla precios contiene el campo `country id` que a su vez tiene un `FOREIGN KEY` que hace referencia a `Productos (id)`
```sql
-- Tabla Productos
CREATE TABLE Productos (
    id INTEGER PRIMARY KEY,
    producto TEXT,
    categoria INTEGER,
    FOREIGN KEY (categoria) REFERENCES Categorias (id)
);

Aqui el resto para que lo analice:

-- Tabla Idiomas
CREATE TABLE Idiomas (id INTEGER PRIMARY KEY, idioma TEXT);

-- Tabla Monedas
CREATE TABLE Monedas (id INTEGER PRIMARY KEY, nombre TEXT, enemonico TEXT);
```

# Inserción de datos:

Insertar datos depende de la palabra `INSERT INTO`

```sql
-- Insertando un unico valor
INSERT INTO Precios (country_id, product_id, price) VALUES (127,3,2.35); 

-- Insertando multiples valores
INSERT INTO Precios (country_id, product_id, price) VALUES 
    (127,3,2.35), (127,4,12.35), (127,5,20.35), (127,6,2.75); 
```
> Note que no se ingresa el `id` porque se autorellena con cada dato nuevo.


## Actualización.

Para actualizar un registro no tiene mucha ciencia, usa la palabra UPDATE para actualizar y la palabra SET para asignar los nuevos valores
```sql
-- Actualizar un registro
UPDATE FROM Precios SET price = 10.35, country_id = 8 WHERE id = 10;
```

Lo anterior es facil, lo dificil es alterar una tabla, debido a que algunas cosas funciona y otras no, la clave aqui son las palabras `ALTER TABLE` que indica la alteracion de una tabla y las operaciones son `ADD, DROP, RENAME`, el problema es que puedes agregar una nueva columna, pero no puedes cambiar el tipo de dato.

Supongamos que te has equivocado y te ha faltado añadir una columna nueva,

```sql
-- Agregar una nueva columna
ALTER TABLE Precios ADD COLUMN categoria;
-- Renombrar una columna
ALTER TABLE Precios RENAME COLUMN categoria  to category_id;
-- Remover una columna
ALTER TABLE Precios DROP COLUMN category_id;
```
Tambien es posible renombrar una tabla, esto es muy comun para hacer una copia de una tabla
```sql
-- Renombra una tabla
ALTER TABLE Precios RENAME TO Precios_Temp;
```
puede ver el esquema en este [vinculo](https://www.sqlite.org/syntax/alter-table-stmt.html)

### Que hacer si haz dañado una tabla.

Si por algun motivo te haz equivocado al agregar una columna, no le haz puesto el tipo de dato que corresponde, te toca rehacer de nuevo la tabla. Esto no es problema si aun no has insertado registros, simplemente borra la tabla y vuelvela a hacer. El problema es cuando ya existen registros.

1. Renombra la tabla erronea como Nombre_Temp `ALTER TABLE Tabla_X TO Tabla_X_Temp`
2. Crea una nueva tabla con el nombre antiguo `CREATE TABLE Tabla_X` asegurandose que las columnas sean del tipo correspondiente.
3. Copia el contenido de la tabla tempora en la tabla nueva `INSERT INTO Table_X (columna_a,columna_b,columna_c) SELECT columna_a, columna_b, columna_c FROM Table_X_Temp;`

## Borrado.

```sql
-- Para borrar una tabla
DROP TABLE Precios;

-- Para borrar un registro
DELETE FROM Precios WHERE id = 10;
```

In [22]:
import sqlite3

# Conectar a la base de datos
conn = sqlite3.connect('PreciosMundi.db')

# Crear un cursor para ejecutar consultas
cur = conn.cursor()

# Ejecutar una consulta SQL
cur.execute('SELECT * FROM Precios WHERE price > (SELECT AVG(price) FROM Precios) LIMIT 5')

# Obtener los resultados de la consulta
results = cur.fetchall()

# Mostrar los resultados
for row in results:
    print(row)

# Cerrar la conexión
conn.close()

(28, 127, 28, 27782.37)
(36, 127, 36, 1587.56)
(37, 127, 37, 1256.82)
(38, 127, 38, 2513.64)
(39, 127, 39, 1852.16)
